<a href="https://colab.research.google.com/github/Leila828/DeepLearning_projects/blob/master/%5BDevMena%5DChat_with_your_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the library for handling Colab secrets
from google.colab import output

# Prompt the user to input the Hugging Face token securely
output.enable_custom_widget_manager()

import getpass

# Prompt user securely (this will hide input while typing)
huggingface_token = getpass.getpass('Enter your Hugging Face API token: ')

Enter your Hugging Face API token: ··········


In [ ]:
!pip show faiss-gpu
!pip install langchain-community
!pip install sentence-transformers
!pip install pypdf langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/2

In [ ]:
from flask import Flask, request, render_template

#from data.dataprovider import key, hg_key
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain_community.llms import HuggingFaceHub
import re

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFaceHub
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import PyPDFLoader
from sklearn.metrics.pairwise import cosine_similarity
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.prompts import PromptTemplate


In [ ]:
llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "top_p": 0.2,
        "presence_penalty": 0.4,
        "frequency_penalty": 0.2,
        "repetition_penalty": 1.03,
    },
    huggingfacehub_api_token=huggingface_token  # Token entered securely
)

<ipython-input-5-7688cc76f137>:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [ ]:
# Define your rag chatbot function
def chat_with_rag(url,query,pdf):
    all_docs = []
    for url in urls:
            loader = WebBaseLoader(url)
            data = loader.load()
            all_docs.extend(data)

    loader = PyPDFLoader(pdf)
    data_pdf = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_documents(all_docs)

    text_splitter_pdf = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts_pdf = text_splitter_pdf.split_documents(data_pdf)

    all_texts = texts + texts_pdf

    embeddings = HuggingFaceEmbeddings()
    db = FAISS.from_documents(all_texts, embeddings)
    retriever = db.as_retriever()

    template = """Based only on the following context, provide a concise answer to the question.Don’t justify your answers. Don’t give information not mentioned in the CONTEXT INFORMATION. If the answer is not available in the context, respond with "I don't know. "

Context: {context}

Question: {question}
Answer:
    """
    prompt = ChatPromptTemplate.from_template(template)


    chain_type_kwargs = {"prompt": prompt}
    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,chain_type_kwargs=chain_type_kwargs)


    # Test the chatbot
    response = chain.run(query)
    # Process and clean the response
    answer = response.split("Answer:")[-1].strip()


    # Return the concise question and answer
    return f"Question: {query}\nAnswer: { answer}"
    #print(response)



In [ ]:
urls = [
    "https://en.wikipedia.org/wiki/FIFA_World_Cup",
    "https://en.wikipedia.org/wiki/2026_FIFA_World_Cup",
]
pdf='/content/fifa.pdf'

In [ ]:

query1= 'who won the last world cup '

In [ ]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.7 MB/s eta 0:00:00


In [ ]:
chat_with_rag(urls, query1 ,pdf)

<ipython-input-6-6127ce876971>:20: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


'Question: who won the last world cup \nAnswer: <|assistant|>\nArgentina won the last World Cup in 2022. However, the given context only goes up until the 2022 tournament, as it mentions that Argentina won their third title at the 2022 tournament. Therefore, based solely on the provided context, we cannot say for certain who will win the next World Cup.'